In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1591563931243_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_cow = spark.read.parquet('s3://data-eng-capstone-cf/production/dim_countries_of_the_world')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_cow.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

227

In [5]:
df_cow.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- gdp_dol_per_capita: integer (nullable = true)

In [6]:
df_cow.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+----------+------------------+
|          country|              region|population|gdp_dol_per_capita|
+-----------------+--------------------+----------+------------------+
|      Afghanistan|ASIA (EX. NEAR EAST)|  31056997|               700|
|          Albania|      EASTERN EUROPE|   3581655|              4500|
|          Algeria|     NORTHERN AFRICA|  32930091|              6000|
|   American Samoa|             OCEANIA|     57794|              8000|
|          Andorra|      WESTERN EUROPE|     71201|             19000|
|           Angola|  SUB-SAHARAN AFRICA|  12127071|              1900|
|         Anguilla| LATIN AMER. & CARIB|     13477|              8600|
|Antigua & Barbuda| LATIN AMER. & CARIB|     69108|             11000|
|        Argentina| LATIN AMER. & CARIB|  39921833|             11200|
|          Armenia| C.W. OF IND. STATES|   2976372|              3500|
|            Aruba| LATIN AMER. & CARIB|     71891|             28000|
|     

In [7]:
df_fact = spark.read.parquet('s3://data-eng-capstone-cf/production/fact_i94')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_fact.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- i94bir: integer (nullable = true)
 |-- visatype: string (nullable = true)
 |-- visitor_count: long (nullable = true)
 |-- country_fk: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94res: string (nullable = true)

In [9]:
df_fact.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

740595

In [10]:
df_fact.select(F.sum('visitor_count')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|sum(visitor_count)|
+------------------+
|          40790529|
+------------------+

In [11]:
df_fact.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-------------+------------+-------+--------+------+-----------+
|i94bir|visatype|visitor_count|  country_fk|i94mode| i94visa|i94mon|     i94res|
+------+--------+-------------+------------+-------+--------+------+-----------+
|    41|      E2|            7|Korea, South|   Land|Business|     4|SOUTH KOREA|
|    45|      F1|            6|Korea, South|    Air| Student|     4|SOUTH KOREA|
|    21|      B2|            8|Korea, South|    Air|Pleasure|     4|SOUTH KOREA|
|    17|      WB|            5|Korea, South|    Air|Business|     4|SOUTH KOREA|
|    74|      WB|            4|Korea, South|    Air|Business|     4|SOUTH KOREA|
+------+--------+-------------+------------+-------+--------+------+-----------+
only showing top 5 rows

In [12]:
# I-94 Arrivals by Country of Residence (COR) and Month
# Exludes Mexico to focus on Overseas countries
df_analysis = df_fact.filter(~df_fact.i94res.contains('MEXICO')) \
                     .groupBy('i94res') \
                     .sum('visitor_count') \
                     .orderBy(F.desc('sum(visitor_count)'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_analysis.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------+
|        i94res|sum(visitor_count)|
+--------------+------------------+
|UNITED KINGDOM|           4587092|
|         JAPAN|           3603786|
|    CHINA, PRC|           3049942|
|       GERMANY|           2046288|
|   SOUTH KOREA|           1982516|
|        BRAZIL|           1725479|
|        FRANCE|           1641152|
|     AUSTRALIA|           1350508|
|         INDIA|           1206771|
|         ITALY|            988860|
|     ARGENTINA|            926321|
|      COLOMBIA|            852618|
|         SPAIN|            807447|
|   NETHERLANDS|            674544|
|     VENEZUELA|            589848|
|        SWEDEN|            560581|
|   SWITZERLAND|            471070|
|        TAIWAN|            466267|
|       IRELAND|            443005|
|        ISRAEL|            425085|
+--------------+------------------+
only showing top 20 rows

In [14]:
# Join fact table with dimension table, inner join
df_join = df_fact.join(df_cow, df_fact.country_fk == df_cow.country, how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_analysis2 = df_join.filter(~df_join.i94res.contains('MEXICO')) \
                      .groupBy('country') \
                      .agg(F.sum('visitor_count').alias('visitor_count'),
                           F.avg('population').alias('population'),
                           F.avg('gdp_dol_per_capita').alias('gdp_dol_per_capita')) \
                      .select(F.col('country'),
                              F.col('visitor_count'),
                              F.col('population'),
                              (F.col('visitor_count')/(F.col('population'))*100).alias('vis_cnt_perc_pop'),
                              F.col('gdp_dol_per_capita')) \
                      .orderBy(F.desc('visitor_count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# What is the population for top countries?
# What is the visitor count relative to the country's population?
# What is the GDP per capita for top countries?
df_analysis2.select(F.col('country'),
                    F.format_number('visitor_count',0).alias('visitor_count'),
                    F.format_number('population',0).alias('population'),
                    F.format_number('vis_cnt_perc_pop',4).alias('vis_cnt_perc_pop'),
                    F.format_number('gdp_dol_per_capita',0).alias('gdp_dol_per_capita')) \
            .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------+-------------+----------------+------------------+
|       country|visitor_count|   population|vis_cnt_perc_pop|gdp_dol_per_capita|
+--------------+-------------+-------------+----------------+------------------+
|United Kingdom|    4,587,092|   60,609,153|          7.5683|            27,700|
|         Japan|    3,603,786|  127,463,611|          2.8273|            28,200|
|         China|    3,049,942|1,313,973,713|          0.2321|             5,000|
|       Germany|    2,046,288|   82,422,299|          2.4827|            27,600|
|  Korea, South|    1,982,516|   48,846,823|          4.0586|            17,800|
|        Brazil|    1,725,479|  188,078,227|          0.9174|             7,600|
|        France|    1,641,152|   60,876,136|          2.6959|            27,600|
|     Australia|    1,350,508|   20,264,082|          6.6645|            29,000|
|         India|    1,206,771|1,095,351,995|          0.1102|             2,900|
|         Italy|      988,86